In [81]:
import pandas as pd
import numpy as np
from sklearn.svm import LinearSVC
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.cross_validation import train_test_split

In [82]:
X_body2 = pd.read_csv("./features/2_body_phog_features.csv").iloc[:, 1:]
X_face2 = pd.read_csv("./features/2_facial_lbp_features.csv").iloc[:, 1:]
X_geo2 = pd.read_csv("./features/2_geo_features.csv").iloc[:, 1:]
X_cont2 = pd.read_csv("./features/2_context_features.csv").iloc[:, 1:]
X_cent2 = pd.read_csv("./features/2_CENTRIST.csv").iloc[:, 1:]
Y_valence2 = pd.read_csv("./labels/2_valence.csv")["V"]
Y_arousal2 = pd.read_csv("./labels/2_arousal.csv")["A"]
Y_combined2 = pd.read_csv("./labels/2_combined.csv")["AV"]
X_body2 = X_body2.loc[Y_arousal2.notnull()]
X_face2 = X_face2.loc[Y_arousal2.notnull()]
X_geo2 = X_geo2.loc[Y_arousal2.notnull()]
X_cont2 = X_cont2.loc[Y_arousal2.notnull()]
X_cent2 = X_cent2.loc[Y_arousal2.notnull()]
Y_arousal2 = Y_arousal2.loc[Y_arousal2.notnull()]

print "Dimensions"
print X_cont2.shape
print X_body2.shape
print X_geo2.shape
print X_face2.shape
print X_cent2.shape
print Y_arousal2.shape

Dimensions
(57, 15)
(57, 1700)
(57, 4556)
(57, 208)
(57, 4064)
(57,)


In [83]:
def classify(X, Y, filename):
    print("Classifying: %s" % filename)
    print
    # Split the dataset in two equal parts
    X_train, X_test, y_train, y_test = train_test_split(
        X, Y, test_size=0.5, random_state=0)

    # Set the parameters by cross-validation
    score = 'accuracy'

#     print("# Tuning hyper-parameters for %s" % score)
#     print

    params = {"C": [0.1, 1.0, 10.0, 100.0, 1000.0, 10000.0]}
    clf = GridSearchCV(LinearSVC(random_state=42), params, cv=10, scoring="accuracy")
    clf.fit(X_train, y_train)

#     print("Best parameters set found on development set:")
#     print
#     print(clf.best_params_)
#     print

#     print("Detailed classification report:")
#     print
#     print("The model is trained on the full development set.")
#     print("The scores are computed on the full evaluation set.")
#     print
    y_true, y_pred = y_test, clf.predict(X_test)
    print(classification_report(y_true, y_pred))
    print("Accuracy: %f" % accuracy_score(y_true, y_pred))
    print(confusion_matrix(y_true, y_pred))
    print

    return (y_pred, y_true)

In [84]:
def fusion(X, Y):
    fused_result = 0
    for i in range(len(Y)):
        row = list(newX[i])
        bins = dict((x, row.count(x)) for x in row)
        if(max(bins, key=bins.get) == Y.iloc[i]):
            fused_result += 1
    return float(fused_result) / len(Y_A_cont2)

# 2 Faces: Arousal

In [85]:
Y_A_cont2, y = classify(X_cont2, Y_arousal2, "2_arousal_cont")
Y_A_cent2, y = classify(X_cent2, Y_arousal2, "2_arousal_cent")
Y_A_geo2, y = classify(X_geo2, Y_arousal2, "2_arousal_geo")
Y_A_face2, y = classify(X_face2, Y_arousal2, "2_arousal_lbp")
Y_A_body2, y = classify(X_body2, Y_arousal2, "2_arousal_phog")
# Fused
newX = np.column_stack((Y_A_cont2, Y_A_cent2, Y_A_geo2, Y_A_face2, Y_A_body2))
print "Accuracy after fusion: %f" % fusion(newX, y)

Classifying: 2_arousal_cont

             precision    recall  f1-score   support

          H       0.38      0.90      0.53        10
          L       0.00      0.00      0.00         3
          M       0.80      0.25      0.38        16

avg / total       0.57      0.45      0.39        29

Accuracy: 0.448276
[[ 9  0  1]
 [ 3  0  0]
 [12  0  4]]

Classifying: 2_arousal_cent

             precision    recall  f1-score   support

          H       0.38      0.50      0.43        10
          L       0.00      0.00      0.00         3
          M       0.62      0.62      0.62        16

avg / total       0.48      0.52      0.49        29

Accuracy: 0.517241
[[ 5  0  5]
 [ 2  0  1]
 [ 6  0 10]]

Classifying: 2_arousal_geo

             precision    recall  f1-score   support

          H       0.50      0.20      0.29        10
          L       0.00      0.00      0.00         3
          M       0.56      0.88      0.68        16

avg / total       0.48      0.55      0.48        

# 2 Faces: Valence

In [86]:
X_body2 = pd.read_csv("./features/2_body_phog_features.csv").iloc[:, 1:]
X_face2 = pd.read_csv("./features/2_facial_lbp_features.csv").iloc[:, 1:]
X_geo2 = pd.read_csv("./features/2_geo_features.csv").iloc[:, 1:]
X_cont2 = pd.read_csv("./features/2_context_features.csv").iloc[:, 1:]
X_cent2 = pd.read_csv("./features/2_CENTRIST.csv").iloc[:, 1:]

In [87]:
Y_V_cont2, y = classify(X_cont2, Y_valence2, "2_valence_cont")
Y_V_cent2, y = classify(X_cent2, Y_valence2, "2_valence_cent")
Y_V_geo2, y = classify(X_geo2, Y_valence2, "2_valence_geo")
Y_V_face2, y = classify(X_face2, Y_valence2, "2_valence_lbp")
Y_V_body2, y = classify(X_body2, Y_valence2, "2_valence_phog")
# Fused
newX = np.column_stack((Y_V_cont2, Y_V_cent2, Y_V_geo2, Y_V_face2, Y_V_body2))
print "Accuracy after fusion: %f" % fusion(newX, y)


Classifying: 2_valence_cont

             precision    recall  f1-score   support

          N       0.29      0.22      0.25         9
        Neu       0.26      1.00      0.41         8
          P       0.00      0.00      0.00        21

avg / total       0.12      0.26      0.15        38

Accuracy: 0.263158
[[ 2  7  0]
 [ 0  8  0]
 [ 5 16  0]]

Classifying: 2_valence_cent

             precision    recall  f1-score   support

          N       0.00      0.00      0.00         9
        Neu       0.29      0.50      0.36         8
          P       0.56      0.43      0.49        21

avg / total       0.37      0.34      0.35        38

Accuracy: 0.342105
[[0 4 5]
 [2 4 2]
 [6 6 9]]

Classifying: 2_valence_geo

             precision    recall  f1-score   support

          N       0.30      0.33      0.32         9
        Neu       0.08      0.12      0.10         8
          P       0.73      0.52      0.61        21

avg / total       0.49      0.39      0.43        38

Accur

# 2 Faces: Combined

In [88]:
Y_AV_cont2, y = classify(X_cont2, Y_combined2, "2_combined_cont")
Y_AV_cent2, y = classify(X_cent2, Y_combined2, "2_combined_cent")
Y_AV_geo2, y = classify(X_geo2, Y_combined2, "2_combined_geo")
Y_AV_face2, y = classify(X_face2, Y_combined2, "2_combined_lbp")
Y_AV_body2, y = classify(X_body2, Y_combined2, "2_combined_phog")
# Fused
newX = np.column_stack((Y_AV_cont2, Y_AV_cent2, Y_AV_geo2, Y_AV_face2, Y_AV_body2))
print "Accuracy after fusion: %f" % fusion(newX, y)


Classifying: 2_combined_cont



/usr/local/lib/python2.7/site-packages/sklearn/metrics/classification.py:960: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


             precision    recall  f1-score   support

         HN       0.00      0.00      0.00         4
         HP       0.50      0.20      0.29        10
         LN       0.00      0.00      0.00         0
         MN       0.00      0.00      0.00         5
         MP       0.50      0.09      0.15        11
        Neu       0.00      0.00      0.00         8

avg / total       0.28      0.08      0.12        38

Accuracy: 0.078947
[[0 0 3 0 1 0]
 [0 2 7 0 0 1]
 [0 0 0 0 0 0]
 [0 1 4 0 0 0]
 [0 0 9 1 1 0]
 [0 1 7 0 0 0]]

Classifying: 2_combined_cent

             precision    recall  f1-score   support

         HN       0.00      0.00      0.00         4
         HP       0.40      0.20      0.27        10
         LN       0.00      0.00      0.00         0
         MN       0.00      0.00      0.00         5
         MP       0.31      0.36      0.33        11
        Neu       0.27      0.50      0.35         8

avg / total       0.25      0.26      0.24        38

Accur

# 3 Faces

In [89]:
X_body3 = pd.read_csv("./features/3_body_phog_features.csv").iloc[:, 1:]
X_face3 = pd.read_csv("./features/3_facial_lbp_features.csv").iloc[:, 1:]
X_geo3 = pd.read_csv("./features/3_geo_features.csv").iloc[:, 1:]
X_cont3 = pd.read_csv("./features/3_context_features.csv").iloc[:, 1:]
X_cent3 = pd.read_csv("./features/3_CENTRIST.csv").iloc[:, 1:]
Y_valence3 = pd.read_csv("./labels/3_valence.csv")["V"]
Y_arousal3 = pd.read_csv("./labels/3_arousal.csv")["A"]
Y_combined3 = pd.read_csv("./labels/3_combined.csv")["AV"]
X_body3 = X_body3.loc[Y_arousal3.notnull()]
X_face3 = X_face3.loc[Y_arousal3.notnull()]
X_geo3 = X_geo3.loc[Y_arousal3.notnull()]
X_cont3 = X_cont3.loc[Y_arousal3.notnull()]
X_cent3 = X_cent3.loc[Y_arousal3.notnull()]
Y_arousal3 = Y_arousal3.loc[Y_arousal3.notnull()]

print "Dimensions"
print X_cont3.shape
print X_body3.shape
print X_geo3.shape
print X_face3.shape
print X_cent3.shape
print Y_arousal3.shape

Dimensions
(48, 22)
(48, 2550)
(48, 6834)
(48, 312)
(48, 4064)
(48,)


# 3 Faces: Arousal

In [90]:
Y_A_cont3, y = classify(X_cont3, Y_arousal3, "3_arousal_cont")
Y_A_cent3, y = classify(X_cent3, Y_arousal3, "3_arousal_cent")
Y_A_geo3, y = classify(X_geo3, Y_arousal3, "3_arousal_geo")
Y_A_face3, y = classify(X_face3, Y_arousal3, "3_arousal_lbp")
Y_A_body3, y = classify(X_body3, Y_arousal3, "3_arousal_phog")
# Fused
newX = np.column_stack((Y_A_cont3, Y_A_cent3, Y_A_geo3, Y_A_face3, Y_A_body3))
print "Accuracy after fusion: %f" % fusion(newX, y)


Classifying: 3_arousal_cont

             precision    recall  f1-score   support

          H       0.48      0.91      0.62        11
          L       0.00      0.00      0.00         3
          M       0.67      0.20      0.31        10

avg / total       0.50      0.50      0.41        24

Accuracy: 0.500000
[[10  0  1]
 [ 3  0  0]
 [ 8  0  2]]

Classifying: 3_arousal_cent

             precision    recall  f1-score   support

          H       0.58      0.64      0.61        11
          L       0.00      0.00      0.00         3
          M       0.42      0.50      0.45        10

avg / total       0.44      0.50      0.47        24

Accuracy: 0.500000
[[7 0 4]
 [0 0 3]
 [5 0 5]]

Classifying: 3_arousal_geo

             precision    recall  f1-score   support

          H       0.57      0.36      0.44        11
          L       0.00      0.00      0.00         3
          M       0.53      0.90      0.67        10

avg / total       0.48      0.54      0.48        24

Accur

# 3 Faces: Valence

In [91]:
X_body3 = pd.read_csv("./features/3_body_phog_features.csv").iloc[:, 1:]
X_face3 = pd.read_csv("./features/3_facial_lbp_features.csv").iloc[:, 1:]
X_geo3 = pd.read_csv("./features/3_geo_features.csv").iloc[:, 1:]
X_cont3 = pd.read_csv("./features/3_context_features.csv").iloc[:, 1:]
X_cent3 = pd.read_csv("./features/3_CENTRIST.csv").iloc[:, 1:]

In [92]:
Y_V_cont3, y = classify(X_cont3, Y_valence3, "3_valence_cont")
Y_V_cent3, y = classify(X_cent3, Y_valence3, "3_valence_cent")
Y_V_geo3, y = classify(X_geo3, Y_valence3, "3_valence_geo")
Y_V_face3, y = classify(X_face3, Y_valence3, "3_valence_lbp")
Y_V_body3, y = classify(X_body3, Y_valence3, "3_valence_phog")
# Fused
newX = np.column_stack((Y_V_cont3, Y_V_cent3, Y_V_geo3, Y_V_face3, Y_V_body3))
print "Accuracy after fusion: %f" % fusion(newX, y)

Classifying: 3_valence_cont

             precision    recall  f1-score   support

          N       0.32      0.55      0.40        11
        Neu       0.00      0.00      0.00        11
          P       0.43      0.55      0.48        11

avg / total       0.25      0.36      0.29        33

Accuracy: 0.363636
[[6 0 5]
 [8 0 3]
 [5 0 6]]

Classifying: 3_valence_cent

             precision    recall  f1-score   support

          N       0.50      0.27      0.35        11
        Neu       0.00      0.00      0.00        11
          P       0.35      0.73      0.47        11

avg / total       0.28      0.33      0.27        33

Accuracy: 0.333333
[[3 2 6]
 [2 0 9]
 [1 2 8]]

Classifying: 3_valence_geo

             precision    recall  f1-score   support

          N       0.17      0.18      0.17        11
        Neu       0.38      0.27      0.32        11
          P       0.38      0.45      0.42        11

avg / total       0.31      0.30      0.30        33

Accuracy: 0.30

# 3 Faces: Combined

In [93]:
Y_AV_cont3, y = classify(X_cont3, Y_combined3, "3_combined_cont")
Y_AV_cent3, y = classify(X_cent3, Y_combined3, "3_combined_cent")
Y_AV_geo3, y = classify(X_geo3, Y_combined3, "3_combined_geo")
Y_AV_face3, y = classify(X_face3, Y_combined3, "3_combined_lbp")
Y_AV_body3, y = classify(X_body3, Y_combined3, "3_combined_phog")
newX = np.column_stack((Y_AV_cont3, Y_AV_cent3, Y_AV_geo3, Y_AV_face3, Y_AV_body3))
print "Accuracy after fusion: %f" % fusion(newX, y)

Classifying: 3_combined_cont

             precision    recall  f1-score   support

         HN       0.00      0.00      0.00         6
         HP       0.00      0.00      0.00         4
         LN       0.00      0.00      0.00         2
         MN       0.11      0.33      0.17         3
         MP       0.29      1.00      0.45         7
        Neu       0.00      0.00      0.00        11

avg / total       0.07      0.24      0.11        33

Accuracy: 0.242424
[[0 0 0 1 5 0]
 [0 0 0 2 2 0]
 [0 0 0 1 1 0]
 [0 0 0 1 2 0]
 [0 0 0 0 7 0]
 [0 0 0 4 7 0]]

Classifying: 3_combined_cent

             precision    recall  f1-score   support

         HN       0.00      0.00      0.00         6
         HP       0.00      0.00      0.00         4
         LN       0.00      0.00      0.00         2
         MN       0.00      0.00      0.00         3
         MP       0.24      0.71      0.36         7
        Neu       0.17      0.09      0.12        11

avg / total       0.11      0

# 4+ Faces

In [94]:
X_body4 = pd.read_csv("./features/4_body_phog_features.csv").iloc[:, 1:]
X_face4 = pd.read_csv("./features/4_facial_lbp_features.csv").iloc[:, 1:]
X_geo4 = pd.read_csv("./features/4_geo_features.csv").iloc[:, 1:]
X_cont4 = pd.read_csv("./features/4_context_features.csv").iloc[:, 1:]
X_cent4 = pd.read_csv("./features/4_CENTRIST.csv").iloc[:, 1:]
Y_valence4 = pd.read_csv("./labels/4_valence.csv")["V"]
Y_arousal4 = pd.read_csv("./labels/4_arousal.csv")["A"]
Y_combined4 = pd.read_csv("./labels/4_combined.csv")["AV"]
X_body4 = X_body4.loc[Y_arousal4.notnull()]
X_face4 = X_face4.loc[Y_arousal4.notnull()]
X_geo4 = X_geo4.loc[Y_arousal4.notnull()]
X_cont4 = X_cont4.loc[Y_arousal4.notnull()]
X_cent4 = X_cent4.loc[Y_arousal4.notnull()]
Y_arousal4 = Y_arousal4.loc[Y_arousal4.notnull()]

print "Dimensions"
print X_cont4.shape
print X_body4.shape
print X_geo4.shape
print X_face4.shape
print X_cent4.shape
print Y_arousal4.shape

Dimensions
(69, 30)
(69, 3400)
(69, 9112)
(69, 416)
(69, 4064)
(69,)


# 4+ Faces: Arousal

In [95]:
Y_A_cont4, y = classify(X_cont4, Y_arousal4, "4_arousal_cont")
Y_A_cent4, y = classify(X_cent4, Y_arousal4, "4_arousal_cent")
Y_A_geo4, y = classify(X_geo4, Y_arousal4, "4_arousal_geo")
Y_A_face4, y = classify(X_face4, Y_arousal4, "4_arousal_lbp")
Y_A_body4, y = classify(X_body4, Y_arousal4, "4_arousal_phog")
# Fused
newX = np.column_stack((Y_A_cont4, Y_A_cent4, Y_A_geo4, Y_A_face4, Y_A_body4))
print "Accuracy after fusion: %f" % fusion(newX, y)


Classifying: 4_arousal_cont

             precision    recall  f1-score   support

          H       0.60      0.19      0.29        16
          L       0.00      0.00      0.00         0
          M       0.80      0.21      0.33        19

avg / total       0.71      0.20      0.31        35

Accuracy: 0.200000
[[ 3 12  1]
 [ 0  0  0]
 [ 2 13  4]]

Classifying: 4_arousal_cent

             precision    recall  f1-score   support

          H       0.43      0.56      0.49        16
          M       0.50      0.37      0.42        19

avg / total       0.47      0.46      0.45        35

Accuracy: 0.457143
[[ 9  7]
 [12  7]]

Classifying: 4_arousal_geo

             precision    recall  f1-score   support

          H       0.35      0.38      0.36        16
          L       0.00      0.00      0.00         0
          M       0.57      0.42      0.48        19

avg / total       0.47      0.40      0.43        35

Accuracy: 0.400000
[[ 6  4  6]
 [ 0  0  0]
 [11  0  8]]

Classifyin

# 4+ Faces: Valence

In [96]:
X_body4 = pd.read_csv("./features/4_body_phog_features.csv").iloc[:, 1:]
X_face4 = pd.read_csv("./features/4_facial_lbp_features.csv").iloc[:, 1:]
X_geo4 = pd.read_csv("./features/4_geo_features.csv").iloc[:, 1:]
X_cont4 = pd.read_csv("./features/4_context_features.csv").iloc[:, 1:]
X_cent4 = pd.read_csv("./features/4_CENTRIST.csv").iloc[:, 1:]

In [97]:
Y_V_cont4, y = classify(X_cont4, Y_valence4, "4_valence_cont")
Y_V_cent4, y = classify(X_cent4, Y_valence4, "4_valence_cent")
Y_V_geo4, y = classify(X_geo4, Y_valence4, "4_valence_geo")
Y_V_face4, y = classify(X_face4, Y_valence4, "4_valence_lbp")
Y_V_body4, y = classify(X_body4, Y_valence4, "4_valence_phog")
# Fused
newX = np.column_stack((Y_V_cont4, Y_V_cent4, Y_V_geo4, Y_V_face4, Y_V_body4))
print "Accuracy after fusion: %f" % fusion(newX, y)

Classifying: 4_valence_cont

             precision    recall  f1-score   support

          N       0.27      0.21      0.24        14
        Neu       0.38      0.23      0.29        22
          P       0.31      0.53      0.39        17

avg / total       0.33      0.32      0.31        53

Accuracy: 0.320755
[[ 3  1 10]
 [ 7  5 10]
 [ 1  7  9]]

Classifying: 4_valence_cent

             precision    recall  f1-score   support

          N       0.33      0.14      0.20        14
        Neu       0.53      0.36      0.43        22
          P       0.34      0.65      0.45        17

avg / total       0.42      0.40      0.38        53

Accuracy: 0.396226
[[ 2  4  8]
 [ 1  8 13]
 [ 3  3 11]]

Classifying: 4_valence_geo

             precision    recall  f1-score   support

          N       0.12      0.14      0.13        14
        Neu       0.25      0.23      0.24        22
          P       0.35      0.35      0.35        17

avg / total       0.25      0.25      0.25        

In [98]:
# 4+ Faces: Combined

In [99]:
Y_AV_cont4, y = classify(X_cont4, Y_combined4, "4_combined_cont")
Y_AV_cent4, y = classify(X_cent4, Y_combined4, "4_combined_cent")
Y_AV_geo4, y = classify(X_geo4, Y_combined4, "4_combined_geo")
Y_AV_face4, y = classify(X_face4, Y_combined4, "4_combined_lbp")
Y_AV_body4, y = classify(X_body4, Y_combined4, "4_combined_phog")
newX = np.column_stack((Y_AV_cont4, Y_AV_cent4, Y_AV_geo4, Y_AV_face4, Y_AV_body4))
print "Accuracy after fusion: %f" % fusion(newX, y)

Classifying: 4_combined_cont

             precision    recall  f1-score   support

         HN       0.00      0.00      0.00         3
         HP       0.00      0.00      0.00         8
         LN       0.00      0.00      0.00         1
         MN       0.00      0.00      0.00        10
         MP       0.00      0.00      0.00         9
        Neu       0.38      0.82      0.51        22

avg / total       0.16      0.34      0.21        53

Accuracy: 0.339623
[[ 0  0  0  0  0  3]
 [ 0  0  0  0  0  8]
 [ 0  0  0  0  0  1]
 [ 0  0  0  0  0 10]
 [ 1  0  0  0  0  8]
 [ 1  3  0  0  0 18]]

Classifying: 4_combined_cent

             precision    recall  f1-score   support

         HN       0.00      0.00      0.00         3
         HP       0.40      0.75      0.52         8
         LN       0.00      0.00      0.00         1
         MN       0.00      0.00      0.00        10
         MP       0.20      0.33      0.25         9
        Neu       0.47      0.32      0.38     